In [19]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind


In [20]:
# Use this dictionary to map state names to two let  ter acronyms

states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}


In [25]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
   
    df=pd.read_fwf('university_towns.txt',names=['RegionName'])
    df.replace(r"\(.*\)","")
    df["RegionName"]=df["RegionName"].str.split('(').str[0]
    df["State"]=(df[df["RegionName"].str.split('[').str[1]=="edit]"])
    df["State"]=df["State"].str.split('[').str[0]
    df["State"].fillna(method="ffill",inplace=True)
    a=df[df["RegionName"].str.contains("edit")]
    df.drop(a.index,inplace=True)
    df.reset_index(inplace=True)
    df["RegionName"]=df["RegionName"].str.rstrip()
    #df.replace(to_replace=states.values(),value=states.keys(),inplace=True)

    return df[["State","RegionName"]]

get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
...,...,...
512,Wisconsin,River Falls
513,Wisconsin,Stevens Point
514,Wisconsin,Waukesha
515,Wisconsin,Whitewater


In [26]:
def gdp_read():
    gdp=pd.read_excel('gdplev.xls',skiprows=219,names=['Current-Dollar and "Real" Gross Domestic Product','GDP in billions of current dollars','gdp in billions of chained 2009 dollars','nan','Quarterly(Seasonally adjusted annual rates)','GDP in billions of current dollars','GDP in billions of chained 2009 dollars','nan2'])
    #pd.set_option('display.max_rows',None)
    gdp.drop(['Current-Dollar and "Real" Gross Domestic Product','GDP in billions of current dollars','gdp in billions of chained 2009 dollars','nan','GDP in billions of current dollars','nan2'],axis=1,inplace=True)
    gdp["diff"]=gdp['GDP in billions of chained 2009 dollars'].diff(periods=1)
    return gdp
gdp_read()

,Quarterly(Seasonally adjusted annual rates),GDP in billions of current dollars.1,GDP in billions of chained 2009 dollars,diff
0,2000q1,10031.0,12359.1,NaN
1,2000q2,10278.3,12592.5,233.4
2,2000q3,10357.4,12607.7,15.2
3,2000q4,10472.3,12679.3,71.6
4,2001q1,10508.1,12643.3,-36.0
...,...,...,...,...
61,2015q2,17998.3,16374.2,105.2
62,2015q3,18141.9,16454.9,80.7
63,2015q4,18222.8,16490.7,35.8
64,2016q1,18281.6,16525.0,34.3


In [27]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    gdp=gdp_read()
    #gdp["diff"]=gdp['GDP in billions of chained 2009 dollars'].diff(periods=1)
    for i in range(len(gdp)-1):
                  if gdp["diff"][i]<np.float64(0) and gdp["diff"][i+1]<np.float64(0):
                          return gdp['Quarterly(Seasonally adjusted annual rates)'][i]
        
    
get_recession_start()

'2008q3'

In [28]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    gdp=gdp_read()
    a=get_recession_start()#getting start of recession 
    b=gdp.set_index('Quarterly(Seasonally adjusted annual rates)').index.get_loc(a)
    for i in range(b,len(gdp)-1):
         if gdp["diff"][i]>0 and gdp["diff"][i+1]>00 :
                return gdp['Quarterly(Seasonally adjusted annual rates)'][i+1]
        
   
get_recession_end()

'2009q4'

In [8]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    gdp=gdp_read()
    start_index=gdp.set_index('Quarterly(Seasonally adjusted annual rates)').index.get_loc(get_recession_start())
    end_index=gdp.set_index('Quarterly(Seasonally adjusted annual rates)').index.get_loc(get_recession_end())
    min_index=gdp['GDP in billions of chained 2009 dollars'][start_index:end_index].idxmin()
        
    return gdp['Quarterly(Seasonally adjusted annual rates)'][min_index]
   
            
get_recession_bottom()

'2009q2'

In [12]:
def convert_housing_data_to_quarters():
    house_data=pd.read_csv('City_Zhvi_AllHomes.csv')
    house_data.drop(house_data.columns[house_data.columns.get_loc('1996-04'):house_data.columns.get_loc('2000-01')],axis=1,inplace=True)
    house_data.drop(house_data.columns[[0,3,4,5]],axis=1,inplace=True)
    house_data.replace(states,regex=True,inplace=True)
    house_data.set_index(["State","RegionName"],inplace=True)
    hdata=house_data.groupby(np.arange(len(house_data.columns))//3,axis=1).mean()
    hdata.columns=gdp_read()["Quarterly(Seasonally adjusted annual rates)"].tolist() + ['2016q3']
    return hdata.sort_index(level=0)
convert_housing_data_to_quarters()
    

2000q1         2000q2         2000q3  \
State   RegionName                                                 
Alabama Adamsville    69033.333333   69166.666667   69800.000000   
        Alabaster    122133.333333  123066.666667  123166.666667   
        Albertville   73966.666667   72600.000000   72833.333333   
        Arab          83766.666667   81566.666667   81333.333333   
        Ardmore                NaN            NaN            NaN   
...                            ...            ...            ...   
Wyoming Burns        101533.333333  104566.666667  108366.666667   
        Casper        89233.333333   89600.000000   89733.333333   
        Cheyenne     116866.666667  120033.333333  121533.333333   
        Evansville   128033.333333  128766.666667  130833.333333   
        Pine Bluffs   93733.333333   95066.666667   94633.333333   

                            2000q4         2001q1         2001q2  \
State   RegionName                                                 
Alabama Adamsville    71966.666667   73466.666667   74000.000000   
        Alabaster    123700.000000  123233.333333  125133.333333   
        Albertville   74200.000000   75900.000000   76000.000000   
        Arab          82966.666667   84200.000000   84533.333333   
        Ardmore                NaN            NaN            NaN   
...                            ...            ...            ...   
Wyoming Burns        113000.000000  115833.333333  117200.000000   
        Casper        93166.666667   95500.000000   97633.333333   
        Cheyenne     123633.333333  125533.333333  126300.000000   
        Evansville   132066.666667  130566.666667  131433.333333   
        Pine Bluffs   98066.666667  103233.333333  104600.000000   

                            2001q3         2001q4         2002q1  \
State   RegionName                                                 
Alabama Adamsville    73333.333333   73100.000000   73333.333333   
        Alabaster    127766.666667  127200.000000  127300.000000   
        Albertville   72066.666667   73566.666667   76533.333333   
        Arab          81666.666667   83900.000000   87266.666667   
        Ardmore                NaN            NaN            NaN   
...                            ...            ...            ...   
Wyoming Burns        117800.000000  117633.333333  117333.333333   
        Casper        99433.333333  100633.333333  101733.333333   
        Cheyenne     126466.666667  128133.333333  128466.666667   
        Evansville   132400.000000  133466.666667  133300.000000   
        Pine Bluffs  106500.000000  104066.666667  102233.333333   

                            2002q2  ...         2014q2         2014q3  \
State   RegionName                  ...                                 
Alabama Adamsville    73133.333333  ...   77066.666667   75966.666667   
        Alabaster    128000.000000  ...  147133.333333  147633.333333   
        Albertville   76366.666667  ...   84033.333333   84766.666667   
        Arab          87700.000000  ...  113366.666667  111700.000000   
        Ardmore                NaN  ...  140533.333333  139566.666667   
...                            ...  ...            ...            ...   
Wyoming Burns        117233.333333  ...  168866.666667  161933.333333   
        Casper       101533.333333  ...  175766.666667  177300.000000   
        Cheyenne     129633.333333  ...  177466.666667  176733.333333   
        Evansville   131066.666667  ...  296733.333333  305666.666667   
        Pine Bluffs  103566.666667  ...  148666.666667  154366.666667   

                            2014q4         2015q1         2015q2  \
State   RegionName                                                 
Alabama Adamsville    71900.000000   71666.666667   73033.333333   
        Alabaster    148700.000000  148900.000000  149566.666667   
        Albertville   86800.000000   88466.666667   89500.000000   
        Arab         111600.000000  110166.666667  109433.333333   
        Ardmore

In [58]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    
    hdata=convert_housing_data_to_quarters()
    start=hdata.columns[hdata.columns.get_loc(get_recession_start())-1]
    end=get_recession_bottom()
    housedata=hdata[[start,end]]
    b=housedata[start]
    a=housedata[end]
    housedata["diff"] = a-b
    udata=get_list_of_university_towns().set_index(["State","RegionName"])
    uni_town=pd.merge(housedata,udata,how='inner',left_index=True,right_index=True)
    non_uni=housedata.drop(uni_town.index)
    t,p=ttest_ind(non_uni["diff"].dropna(),uni_town["diff"].dropna())
    if p< 0.01:
        different =True
    else:
        different=False
    if non_uni["diff"].mean() < uni["diff"].mean()  :
        better= 'non_university town'
    else:
        better = 'university town'
        
    return (different, p , better})
run_ttest()

<ipython-input-58-89e7a8006991>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  housedata["diff"] = a-b


AttributeError: 'numpy.ndarray' object has no attribute 'dropna'